In [1]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf

from _context import dl_uncertainty
import dl_uncertainty.tf_utils as tf_utils
import dl_uncertainty.tf_utils.layers as layers

In [2]:
x_ = np.array([[1,2],[3,4.0]])
#x = tf.constant(np.array([[1,2],[3,4.0]]), dtype=tf.float32)
x = tf.placeholder(tf.float32, shape=[None, 2])
sess = tf.Session()

## 1. Sampling random operations with no variables

### 1.1. Dropout

In [3]:
@layers.scoped
def dropout(x):
    return tf.nn.dropout(x, keep_prob=0.5)

ys = layers.sample(lambda x: dropout(x, name='do'), x, 100)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x
ys2 = layers.sample(lambda x: dropout(x, name='do'), x, 8, examplewise=True)

In [4]:
sess.run(tf.global_variables_initializer())

print('x')
print(sess.run(x, {x:x_}))

print('')
print('dropout(x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys, {x:x_})
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean, {x:x_})
print(m)

print('')
print('Example-wise dropout(x) sampling (first 2 elements)')
s = sess.run(ys2, {x:x_})
print(s[:2], '...')
print('shape:', s.shape)

x
[[1. 2.]
 [3. 4.]]

dropout(x) sampling ([n,N,...]) (first 3 samples)
[[[0. 0.]
  [0. 0.]]

 [[2. 4.]
  [6. 8.]]

 [[0. 0.]
  [0. 0.]]] ...
shape: (100, 2, 2)

Sample mean (estimation of x)
[[1.   1.88]
 [2.94 3.76]]

Example-wise dropout(x) sampling (first 2 elements)
[[[2. 0.]
  [0. 4.]
  [2. 4.]
  [2. 4.]
  [0. 0.]
  [0. 4.]
  [2. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 8.]
  [0. 0.]
  [6. 0.]
  [6. 8.]
  [6. 8.]
  [6. 0.]
  [0. 0.]]] ...
shape: (2, 8, 2)


### 1.2. `tf.random_*`, `use_tf_loop=False`

In [5]:
@layers.scoped
def random(m, sd):
    return tf.random_normal(shape=tf.shape(m), mean=m, stddev=sd)

ys = layers.sample(lambda x: random(x, x), x, 100, examplewise=False, use_tf_loop=False)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x

In [6]:
print('')
print('random_normal(x, x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys, {x:x_})
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean, {x:x_})
print(m)


random_normal(x, x) sampling ([n,N,...]) (first 3 samples)
[[[ 1.4026287  2.0758417]
  [ 2.9668849  7.358088 ]]

 [[-0.3348403 -0.6469178]
  [-4.731756   3.8538756]]

 [[ 0.743213   5.477846 ]
  [-1.381516   0.214185 ]]] ...
shape: (100, 2, 2)

Sample mean (estimation of x)
[[1.0137873 2.1574497]
 [2.9419515 3.888212 ]]


## 2. Sampling operations with variables

In [7]:
def init_func(*args, **kwargs): return tf.random_uniform([])

@layers.scoped
def rando(x):
    v = tf.get_variable("var", initializer=tf.random_uniform([]))
    #tf.get_variable_scope().reuse_variables()
    print(v)
    return v*tf.random_uniform(tf.shape(x))*tf.identity(x)

In [8]:
#trando = tf.make_template('rando', rando)
#rr = [tf.map_fn(rando, x) for _ in range(2)]
samp = layers.sample(rando, x, 100, use_tf_loop=False)

<tf.Variable 'sample_3/rando/var:0' shape=() dtype=float32_ref>


In [9]:
sess.run(tf.global_variables_initializer())

#rr_ = sess.run(rr)
samp_ = sess.run(samp, {x:x_})
print(samp_)

#for a in rr_:
#    print(a)

[[[0.16678089 0.21101703]
  [1.0974121  0.31738746]]

 [[0.6112743  1.6983688 ]
  [0.3187951  1.2201843 ]]

 [[0.8015221  1.0359783 ]
  [0.7142778  1.4532229 ]]

 [[0.12269711 0.1269303 ]
  [0.5345001  2.6852798 ]]

 [[0.18170227 0.31475967]
  [1.1909847  1.766895  ]]

 [[0.4222304  1.2619742 ]
  [2.3919988  3.4080436 ]]

 [[0.16627802 0.32225615]
  [0.22479723 0.3016074 ]]

 [[0.497767   0.72368675]
  [1.9848232  0.57231104]]

 [[0.80951285 0.40908587]
  [0.9154693  2.8107145 ]]

 [[0.01148906 0.8609089 ]
  [1.6417344  2.4720755 ]]

 [[0.15708703 1.2874364 ]
  [1.6202102  0.4801992 ]]

 [[0.6024425  1.534543  ]
  [0.7193287  0.17067218]]

 [[0.18428841 0.9475484 ]
  [2.3433063  2.5604715 ]]

 [[0.87571675 1.1172875 ]
  [1.7777653  2.6638732 ]]

 [[0.7113558  0.735991  ]
  [2.45219    1.9457842 ]]

 [[0.07290094 0.56689036]
  [0.549971   2.4301202 ]]

 [[0.7137689  1.785468  ]
  [1.2459549  1.7884374 ]]

 [[0.3391053  1.0579593 ]
  [0.6886556  2.6757367 ]]

 [[0.07232685 0.56494164]
  

In [10]:
# samples should be different because outputs come from different subgraphs with different values of "var"
assert np.any(samp_[0] != samp_[1])

In [11]:
v = tf.get_variable("var2", initializer=tf.random_uniform([]))

@layers.scoped
def wrapv(fn, x):
    print(x.shape)
    with tf.control_dependencies([tf.assign(v, x[0,0])]):
        return fn(v)
samp = layers.sample(lambda x: wrapv(rando, x), x, 10)

(20,)


ValueError: Index out of range using input dim 1; input has only 1 dims for 'sample_4/map/while/wrapv/strided_slice' (op: 'StridedSlice') with input shapes: [20], [2], [2], [2] and with computed input tensors: input[3] = <1 1>.

In [18]:
sess.run(tf.global_variables_initializer())
sess.run(samp)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [2,2]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[2,2], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\Ivan\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Ivan\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C